In [19]:
# Importação das bibliotecas necessárias
from dataclasses import dataclass
from typing import Callable, List
from random import randint, sample, random

In [20]:

# Definição da classe Cromossomo para representar os indivíduos
@dataclass
@dataclass
class Cromossomo:
    dados: List[int]
    fitness: float = 0

@dataclass
class Config:
    tam_cromossomo: int
    tam_populacao: int
    fitness: Callable[[Cromossomo], float]
    tam_torneio: int
    taxa_mutacao: float
    max_geracoes: int
    max_estagnacao: int

In [21]:
# Função para inicializar a população com cromossomos aleatórios
def inicializar_populacao(tam_populacao, tam_cromossomo):
    populacao = []
    for _ in range(tam_populacao):
        cromossomo = Cromossomo([randint(0, 1) for _ in range(tam_cromossomo)])
        populacao.append(cromossomo)
    return populacao


# Função de seleção por torneio
def torneio(populacao, tam_torneio):
    casais = []

    for i in range(len(populacao) // 2):
        torneio1 = sample(populacao, tam_torneio)
        pai1 = max(torneio1, key=lambda x: x.fitness)

        torneio2 = sample(populacao, tam_torneio)
        pai2 = max(torneio2, key=lambda x: x.fitness)

        casais.append((pai1, pai2))

    return casais
# Função de crossover com um ponto de corte
def crossover_1_corte(casais):
    filhos = []

    for par in casais:
        pai1 = par[0].dados
        pai2 = par[1].dados

        corte = randint(1, len(pai1) - 1)

        filho1 = pai1[:corte] + pai2[corte:]
        filho2 = pai2[:corte] + pai1[corte:]

        filhos.append(Cromossomo(filho1))
        filhos.append(Cromossomo(filho2))

    return filhos

def mutacao(cromossomos, taxa_mutacao):
    for cromossomo in cromossomos:
        for i, gene in enumerate(cromossomo.dados):
            if random() < taxa_mutacao:
                cromossomo.dados[i] = 1 - gene

def elitismo(populacao):
    populacao.sort(key=lambda x: x.fitness, reverse=True)
    return populacao[:len(populacao) // 2]

In [22]:
# Função principal do Algoritmo Genético

def algoritmo_genetico(config):
    t = 0
    P = inicializar_populacao(config.tam_populacao, config.tam_cromossomo)
    melhor_individuo = max(P, key=lambda x: x.fitness)
    contagem_estagnacao = 0

    while t < config.max_geracoes and contagem_estagnacao < config.max_estagnacao:
        for cromossomo in P:
            cromossomo.fitness = config.fitness(cromossomo)

        pais = torneio(P, config.tam_torneio)
        filhos = crossover_1_corte(pais)
        mutacao(filhos, config.taxa_mutacao)
        for filho in filhos:
            filho.fitness = config.fitness(filho)

        P = elitismo(P + filhos)

        melhor_atual = max(P, key=lambda x: x.fitness)

        if melhor_atual.fitness > melhor_individuo.fitness:
            melhor_individuo = melhor_atual
            contagem_estagnacao = 0
        else:
            contagem_estagnacao += 1

        t += 1

    return melhor_individuo

In [23]:
# Função de avaliação do problema: Maximizar a quantidade de 1s no cromossomo
def maximizar_1s(cromossomo):
    return sum(cromossomo.dados)

# Parâmetros do AG
config = Config(
    tam_cromossomo=100,
    tam_populacao=100,
    fitness=maximizar_1s,
    tam_torneio=10,
    taxa_mutacao=0.01,
    max_geracoes=1000,
    max_estagnacao=100
)

# Execução do AG
melhor_solucao = algoritmo_genetico(config)
print("Melhor solução encontrada:", melhor_solucao.dados)
print("Valor do fitness da melhor solução:", melhor_solucao.fitness)

Melhor solução encontrada: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Valor do fitness da melhor solução: 100
